<a href="https://colab.research.google.com/github/degar405/ExperimentacoesPreDiagnosticoGlaucoma/blob/main/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-image

**Cálculo de mapas de textura**

In [85]:
from google.colab import drive
from os import listdir
from skimage import io
import pywt
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix

drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

def exibirImagensCoeficientesWavelet(coeffs):
  titles = ['Approximation', ' Horizontal detail',
          'Vertical detail', 'Diagonal detail']
  LL, (LH, HL, HH) = coeffs

  fig = plt.figure(figsize=(12, 3))
  for i, a in enumerate([LL, LH, HL, HH]):
      ax = fig.add_subplot(1, 4, i + 1)
      ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
      ax.set_title(titles[i], fontsize=10)
      ax.set_xticks([])
      ax.set_yticks([])

  fig.tight_layout()
  plt.show()

def separarCanaisRGB(imagem):
  canalR = []
  canalG = []
  canalB = []

  for linha in imagem:
    linhaR = []
    linhaG = []
    linhaB = []

    for pixel in linha:
      linhaR.append(pixel[0])
      linhaG.append(pixel[1])
      linhaB.append(pixel[2])

    canalR.append(linhaR)
    canalG.append(linhaG)
    canalB.append(linhaB)
  
  return (np.array(canalR), np.array(canalG), np.array(canalB))

def aplicarWavelet(imagem, implementacao):
  return pywt.dwt2(imagem, implementacao)

def combinarImagensEmTonsDeCinza(R, G, B):
  imagem = []
  quantidadeLinhas = len(R)
  quantidadeColunas = len(R[0])

  for nLinha in range(0, quantidadeLinhas):
    linha = []

    for nColuna in range(0, quantidadeColunas):
      pixel = round(0.2126 * R[nLinha][nColuna] + 0.7154 * G[nLinha][nColuna] + 0.0721 * B[nLinha][nColuna])
      linha.append(pixel)

    imagem.append(linha)

  return imagem

def combinarCoeficientesWavelet(coeficientesR, coeficientesG, coeficientesB):
  (LLR, (LHR, HLR, HHR)) = coeficientesR
  (LLG, (LHG, HLG, HHG)) = coeficientesG
  (LLB, (LHB, HLB, HHB)) = coeficientesB

  LL = combinarImagensEmTonsDeCinza(LLR, LLG, LLB)
  LH = combinarImagensEmTonsDeCinza(LHR, LHG, LHB)
  HL = combinarImagensEmTonsDeCinza(HLR, HLG, HLB)
  HH = combinarImagensEmTonsDeCinza(HHR, HHG, HHB)

  return (LL, (LH, HL, HH))

def aplicarGlcm(imagem):
  distancias = [1]
  direcoes = [0, np.pi/4, np.pi/2, 3*np.pi/4]
  resultadoGlcm = graycomatrix(np.array(imagem, dtype=np.uint8), distancias, direcoes)

  qtdDirecoes = len(direcoes)
  matrizes = []
  for i in range(0, qtdDirecoes):
    matrizes.append(resultadoGlcm[:, :, 0, i])

  niveis = len(matrizes[0])
  matrizRetorno = []
  for nLinha in range(0, niveis):
    linha = []
    for nColuna in range(0, niveis):
      acumuladorFrequencia = 0
      for matriz in matrizes:
        acumuladorFrequencia += matriz[nLinha, nColuna]
      linha.append(round(acumuladorFrequencia/qtdDirecoes))
    matrizRetorno.append(linha)

  matrizRetorno = np.array(matrizRetorno)
  return matrizRetorno

def aplicarDescritoresDeTextura(imagem, implementacaoWavelet):
  (canalR, canalG, canalB) = separarCanaisRGB(imagem)

  coeficientesWaveletR = aplicarWavelet(canalR, implementacaoWavelet)
  coeficientesWaveletG = aplicarWavelet(canalG, implementacaoWavelet)
  coeficientesWaveletB = aplicarWavelet(canalB, implementacaoWavelet)

  coeficientesWavelet = combinarCoeficientesWavelet(coeficientesWaveletR, coeficientesWaveletG, coeficientesWaveletB)

  #exibirImagensCoeficientesWavelet(coeficientesWavelet)
  LL, (LH , HL, HH) = coeficientesWavelet
  return (aplicarGlcm(LL), aplicarGlcm(LH), aplicarGlcm(HL), aplicarGlcm(HH))

def calcularAcc(resultadoClassificador, resultado):
  corretos = 0
  for i in range(len(resultado)):
    if resultadoClassificador[i] == resultado[i]:
      corretos += 1

  return float(corretos) / len(resultado)

def unir4MatrizesNxN(m1, m2, m3, m4):
  matrizSuperior = []
  matrizInferior = []

  for i in range(len(m1)):
    matrizSuperior.append(m1[i].tolist() + m2[i].tolist())
    matrizInferior.append(m3[i].tolist() + m4[i].tolist())

  return matrizSuperior + matrizInferior

def normalizarMatriz(matriz):
  npMatriz = np.matrix(matriz)
  mx, mn = npMatriz.max(), npMatriz.min()
  n = len(matriz)
  qtd = (mx - mn + 1)
  diff = 255 / qtd

  resultado = []
  for i in range(n):
    linha = []
    for j in range(n):
      linha.append(int(diff * matriz[i][j]))
    resultado.append(linha)

  return resultado

def gerarMatrizFormatoResnet(matriz):
  matrizResultado = []
  for l in matriz:
    linha = []
    for e in l:
      linha.append([e,e,e])
    matrizResultado.append(linha)

  return np.array(matrizResultado)

def TesteMapasDeTextura():
  imagemTeste = io.imread(root_path + "Normal/Im001.jpg")
  io.imshow(imagemTeste)
  io.show()

  LL, LH, HL, HH = aplicarDescritoresDeTextura(imagemTeste, "haar")
  matriz = unir4MatrizesNxN(LL, LH, HL, HH)
  matrizNormalizada = normalizarHistograma(matriz)

  print(len(matrizNormalizada))
  print(len(matrizNormalizada[0]))
  print(matriz[0])
  print(matrizNormalizada[0])
  print(matriz[511])
  print(matrizNormalizada[511])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Geração casos de treinamento e de teste Mapas de Textura**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

amostrasNormaisMapasTextura = []
for i in range(1, 256):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Normal/" + nomeArquivo)
  LL, LH, HL, HH = aplicarDescritoresDeTextura(imagem, "haar")
  amostrasNormaisMapasTextura.append(LL.flatten() + LH.flatten() + HL.flatten() + HH.flatten())

amostrasGlaucomatosasMapasTextura = []
for i in range(256, 456):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Glaucoma and glaucoma suspicious/" + nomeArquivo)
  LL, LH, HL, HH = aplicarDescritoresDeTextura(imagem, "haar")
  amostrasGlaucomatosasMapasTextura.append(LL.flatten() + LH.flatten() + HL.flatten() + HH.flatten())

In [ ]:
kernel = 1.0 * RBF(1.0)
gpcMapasTextura = GaussianProcessClassifier(kernel=kernel, random_state=0)

quotaTreinamentoNormaisMapasTextura = round(0.8 * len(amostrasNormaisMapasTextura))
quotaTreinamentoGlaucomatosasMapasTextura = round(0.8 * len(amostrasGlaucomatosasMapasTextura))
amostrasMapasTextura = amostrasNormaisMapasTextura[:quotaTreinamentoNormaisMapasTextura] + amostrasGlaucomatosasMapasTextura[:quotaTreinamentoGlaucomatosasMapasTextura]
saidaAmostrasMapasTextura = [0 for i in range(quotaTreinamentoNormaisMapasTextura)] + [1 for i in range(quotaTreinamentoGlaucomatosasMapasTextura)]
gpcMapasTextura.fit(amostrasMapasTextura, saidaAmostrasMapasTextura)

amostrasTesteMapasTextura = amostrasNormaisMapasTextura[quotaTreinamentoNormaisMapasTextura:] + amostrasGlaucomatosasMapasTextura[quotaTreinamentoGlaucomatosasMapasTextura:]
saidaAmostrasTesteMapasTextura = [0 for i in range(len(amostrasNormaisMapasTextura) - quotaTreinamentoNormaisMapasTextura)] + [1 for i in range(len(amostrasGlaucomatosasMapasTextura) - quotaTreinamentoGlaucomatosasMapasTextura)]

predicaoTesteMapasTextura = gpcMapasTextura.predict(amostrasTesteMapasTextura)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.5604395604395604


In [ ]:
!pip install tensorflow


**Geração casos de treinamento e de teste Resnet50 com imagem pura**

In [ ]:
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import gridspec

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.callbacks import ModelCheckpoint

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

model = ResNet50(weights = 'imagenet', include_top = False)

root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

amostrasNormaisResnet = []
for i in range(1, 256):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Normal/" + nomeArquivo)
  # Convert image to RGB from BGR (another way is to use "image = image[:, :, ::-1]" code)
  imagem = cv.cvtColor(imagem, cv.COLOR_BGR2RGB)

  # Resize image to 224x224 size
  image = cv.resize(imagem, (224, 224)).reshape(-1, 224, 224, 3)

  # We need to preprocess imageto fulfill ResNet50 requirements
  image = preprocess_input(image)

  # Extracting our features
  features = model.predict(image)
  amostrasNormaisResnet.append(features.flatten())

amostrasGlaucomatosasResnet = []
for i in range(256, 456):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Glaucoma and glaucoma suspicious/" + nomeArquivo)
  # Convert image to RGB from BGR (another way is to use "image = image[:, :, ::-1]" code)
  imagem = cv.cvtColor(imagem, cv.COLOR_BGR2RGB)

  # Resize image to 224x224 size
  image = cv.resize(imagem, (224, 224)).reshape(-1, 224, 224, 3)

  # We need to preprocess imageto fulfill ResNet50 requirements
  image = preprocess_input(image)

  # Extracting our features
  features = model.predict(image)
  amostrasGlaucomatosasResnet.append(features.flatten())


In [ ]:
gpcResnet = GaussianProcessClassifier()

quotaTreinamentoNormaisResnet = round(0.8 * len(amostrasNormaisResnet))
quotaTreinamentoGlaucomatosasResnet = round(0.8 * len(amostrasGlaucomatosasResnet))
amostrasResnet = amostrasGlaucomatosasResnet[:quotaTreinamentoGlaucomatosasResnet] + amostrasNormaisResnet[:quotaTreinamentoNormaisResnet]
saidaAmostrasResnet = [1 for i in range(quotaTreinamentoGlaucomatosasResnet)] + [0 for i in range(quotaTreinamentoNormaisResnet)]
gpcResnet.fit(amostrasResnet, saidaAmostrasResnet)

amostrasTesteResnet = amostrasNormaisResnet[quotaTreinamentoNormaisResnet:] + amostrasGlaucomatosasResnet[quotaTreinamentoGlaucomatosasResnet:]
saidaAmostrasTesteResnet = [0 for i in range(len(amostrasNormaisResnet) - quotaTreinamentoNormaisResnet)] + [1 for i in range(len(amostrasGlaucomatosasResnet) - quotaTreinamentoGlaucomatosasResnet)]

predicaoTesteResnet = gpcResnet.predict(amostrasTesteResnet)
gpcResnet.predict_proba(amostrasResnet)

**Aplicação de descritores de textura com unificação das matrizes geradas**

In [89]:
root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

mapasTexturaUnificadosNormais = []
for i in range(1, 256):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Normal/" + nomeArquivo)
  LL, LH, HL, HH = aplicarDescritoresDeTextura(imagem, "haar")
  mapasTexturaUnificadosNormais.append(unir4MatrizesNxN(LL, LH, HL, HH))

mapasTexturaUnificadosGlaucomatosas = []
for i in range(256, 456):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Glaucoma and glaucoma suspicious/" + nomeArquivo)
  LL, LH, HL, HH = aplicarDescritoresDeTextura(imagem, "haar")
  mapasTexturaUnificadosGlaucomatosas.append(unir4MatrizesNxN(LL, LH, HL, HH))

**Geração casos de treinamento e de teste WcgResnet**

In [90]:
root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

model = ResNet50(weights = 'imagenet', include_top = False)

amostrasWcgResnetNormais = []
for caso in mapasTexturaUnificadosNormais:
  image = cv.resize(gerarMatrizFormatoResnet(caso), (512, 512)).reshape(-1, 512, 512, 3)
  image = preprocess_input(image)
  features = model.predict(image)
  amostrasWcgResnetNormais.append(features.flatten())

amostrasWcgResnetGlaucomatosas = []
for caso in mapasTexturaUnificadosGlaucomatosas:
  image = cv.resize(gerarMatrizFormatoResnet(caso), (512, 512)).reshape(-1, 512, 512, 3)
  image = preprocess_input(image)
  features = model.predict(image)
  amostrasWcgResnetGlaucomatosas.append(features.flatten())

quotaTreinamentoWcgResnetNormais = round(0.8 * len(amostrasWcgResnetNormais))
quotaTreinamentoWcgResnetGlaucomatosas = round(0.8 * len(amostrasWcgResnetGlaucomatosas))

amostrasWcgResnet = amostrasWcgResnetNormais[:quotaTreinamentoWcgResnetNormais] + amostrasWcgResnetGlaucomatosas[:quotaTreinamentoWcgResnetGlaucomatosas]
saidaAmostrasWcgResnet = [0 for i in range(quotaTreinamentoWcgResnetNormais)] + [1 for i in range(quotaTreinamentoWcgResnetGlaucomatosas)]

amostrasTesteWcgResnet = amostrasWcgResnetNormais[quotaTreinamentoWcgResnetNormais:] + amostrasWcgResnetGlaucomatosas[quotaTreinamentoWcgResnetGlaucomatosas:]
saidaAmostrasTesteWcgResnet = [0 for i in range(len(amostrasWcgResnetNormais) - quotaTreinamentoWcgResnetNormais)] + [1 for i in range(len(amostrasWcgResnetGlaucomatosas) - quotaTreinamentoWcgResnetGlaucomatosas)]

1/1 [==============================] - 1s 759ms/step


**Geração casos de treinamento e de teste WcgNormResnet**

In [92]:
root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

model = ResNet50(weights = 'imagenet', include_top = False)

amostrasWcgNormResnetNormais = []
for caso in mapasTexturaUnificadosNormais:
  image = cv.resize(gerarMatrizFormatoResnet(normalizarMatriz(caso)), (512, 512)).reshape(-1, 512, 512, 3)
  image = preprocess_input(image)
  features = model.predict(image)
  amostrasWcgNormResnetNormais.append(features.flatten())

amostrasWcgNormResnetGlaucomatosas = []
for caso in mapasTexturaUnificadosGlaucomatosas:
  image = cv.resize(gerarMatrizFormatoResnet(normalizarMatriz(caso)), (512, 512)).reshape(-1, 512, 512, 3)
  image = preprocess_input(image)
  features = model.predict(image)
  amostrasWcgNormResnetGlaucomatosas.append(features.flatten())

quotaTreinamentoWcgNormResnetNormais = round(0.8 * len(amostrasWcgNormResnetNormais))
quotaTreinamentoWcgNormResnetGlaucomatosas = round(0.8 * len(amostrasWcgNormResnetGlaucomatosas))

amostrasWcgNormResnet = amostrasWcgNormResnetNormais[:quotaTreinamentoWcgNormResnetNormais] + amostrasWcgNormResnetGlaucomatosas[:quotaTreinamentoWcgNormResnetGlaucomatosas]
saidaAmostrasWcgNormResnet = [0 for i in range(quotaTreinamentoWcgNormResnetNormais)] + [1 for i in range(quotaTreinamentoWcgNormResnetGlaucomatosas)]

amostrasTesteWcgNormResnet = amostrasWcgNormResnetNormais[quotaTreinamentoWcgNormResnetNormais:] + amostrasWcgNormResnetGlaucomatosas[quotaTreinamentoWcgNormResnetGlaucomatosas:]
saidaAmostrasTesteWcgNormResnet = [0 for i in range(len(amostrasWcgNormResnetNormais) - quotaTreinamentoWcgNormResnetNormais)] + [1 for i in range(len(amostrasWcgNormResnetGlaucomatosas) - quotaTreinamentoWcgNormResnetGlaucomatosas)]

1/1 [==============================] - 1s 752ms/step


**WCG + DT**

In [88]:
from sklearn import tree
from sklearn.metrics import cohen_kappa_score

def executarWcgDt():
  dtMapasTextura = tree.DecisionTreeClassifier()
  dtMapasTextura = dtMapasTextura.fit(amostrasMapasTextura, saidaAmostrasMapasTextura)
  predicaoDtMapasTextura = dtMapasTextura.predict(amostrasTesteMapasTextura)
  return (calcularAcc(predicaoDtMapasTextura,saidaAmostrasTesteMapasTextura), cohen_kappa_score(predicaoDtMapasTextura, saidaAmostrasTesteMapasTextura))

accWcgDt = 0
kappaWcgDt = 0
for i in range(10):
  acc, kappa = executarWcgDt()
  accWcgDt += acc
  kappaWcgDt += kappa

print(accWcgDt/10)
print(kappaWcgDt/10)

0.6835164835164835
0.3748461308441102


**Resnet50 + DT**

In [87]:
from sklearn import tree
from sklearn.metrics import cohen_kappa_score

def executarResnetDt():
  dtResnet = tree.DecisionTreeClassifier()
  dtResnet = dtResnet.fit(amostrasResnet, saidaAmostrasResnet)
  predicaoDtResnet = dtResnet.predict(amostrasTesteResnet)
  return (calcularAcc(predicaoDtResnet,saidaAmostrasTesteResnet), cohen_kappa_score(predicaoDtResnet, saidaAmostrasTesteResnet))

accResnetDt = 0
kappaResnetDt = 0
for i in range(10):
  acc, kappa = executarResnetDt()
  accResnetDt += acc
  kappaResnetDt += kappa

print(accResnetDt/10)
print(kappaResnetDt/10)

0.7527472527472527
0.5111744858562731


**WCG + Resnet50 + DT**

In [91]:
from sklearn import tree
from sklearn.metrics import cohen_kappa_score

def executarWcgResnetDt():
  dtWcgResnet = tree.DecisionTreeClassifier()
  dtWcgResnet = dtWcgResnet.fit(amostrasWcgResnet, saidaAmostrasWcgResnet)
  predicaoDtWcgResnet = dtWcgResnet.predict(amostrasTesteWcgResnet)
  return (calcularAcc(predicaoDtWcgResnet,saidaAmostrasTesteWcgResnet), cohen_kappa_score(predicaoDtWcgResnet, saidaAmostrasTesteWcgResnet))

accWcgResnetDt = 0
kappaWcgResnetDt = 0
for i in range(10):
  acc, kappa = executarWcgResnetDt()
  accWcgResnetDt += acc
  kappaWcgResnetDt += kappa

print(accWcgResnetDt/10)
print(kappaWcgResnetDt/10)

0.6670329670329672
0.3263401886416099


**WCG Normalizado + Resnet50 + DT**

In [93]:
from sklearn import tree
from sklearn.metrics import cohen_kappa_score

def executarWcgNormResnetDt():
  dtWcgNormResnet = tree.DecisionTreeClassifier()
  dtWcgNormResnet = dtWcgNormResnet.fit(amostrasWcgNormResnet, saidaAmostrasWcgNormResnet)
  predicaoDtWcgNormResnet = dtWcgNormResnet.predict(amostrasTesteWcgNormResnet)
  return (calcularAcc(predicaoDtWcgNormResnet,saidaAmostrasTesteWcgNormResnet), cohen_kappa_score(predicaoDtWcgNormResnet, saidaAmostrasTesteWcgNormResnet))

accWcgNormResnetDt = 0
kappaWcgNormResnetDt = 0
for i in range(10):
  acc, kappa = executarWcgNormResnetDt()
  accWcgNormResnetDt += acc
  kappaWcgNormResnetDt += kappa

print(accWcgNormResnetDt/10)
print(kappaWcgNormResnetDt/10)

0.7252747252747254
0.4359730291913454
